In [1]:
%load_ext nb_black
# !pip install nb_black

import warnings

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import metrics as m
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE  # doctest: +NORMALIZE_WHITESPACE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
from sklearn.pipeline import make_pipeline
from mlxtend.feature_selection import ColumnSelector
from sklearn import model_selection
from mlxtend.classifier import StackingClassifier
from thundersvm import SVC as svmgpu

randomseed = 7
np.random.seed(randomseed)

Using TensorFlow backend.


<IPython.core.display.Javascript object>

In [5]:
FinalAccuracy = []
FinalPrecision = []
FinalRecall = []
FinalF1 = []

<IPython.core.display.Javascript object>

In [7]:
for cv_i in range(10):

    print(cv_i)

    acc = pd.read_csv("cv_resulsts/CV_" + str(cv_i) + "_acc.txt")
    ytest = pd.read_csv("cv_resulsts/CV_" + str(cv_i) + "_tempytest.txt")

    ypredproba_all_class_0 = pd.read_csv(
        "cv_resulsts/CV_" + str(cv_i) + "_ypredproba_all_class_0.txt"
    ).transpose()
    ypredproba_all_class_1 = pd.read_csv(
        "cv_resulsts/CV_" + str(cv_i) + "_ypredproba_all_class_1.txt"
    ).transpose()
    ypredproba_all_class_2 = pd.read_csv(
        "cv_resulsts/CV_" + str(cv_i) + "_ypredproba_all_class_2.txt"
    ).transpose()
    confmatrix = pd.read_csv("cv_resulsts/CV_" + str(cv_i) + "_confmatrix.txt")

    tempacc = pd.DataFrame(np.array(acc), columns=["acc"])
    indexs = tempacc.index
    len(indexs)

    ypredproba_all_class_0 = ypredproba_all_class_0.iloc[:, indexs]
    ypredproba_all_class_1 = ypredproba_all_class_1.iloc[:, indexs]
    ypredproba_all_class_2 = ypredproba_all_class_2.iloc[:, indexs]

    ypredproba_all_class_0.columns = np.arange(indexs.shape[0])
    ypredproba_all_class_1.columns = np.arange(indexs.shape[0])
    ypredproba_all_class_2.columns = np.arange(indexs.shape[0])

    confmatrix = confmatrix.iloc[indexs, :].reset_index(drop=True)

    class0 = []
    class1 = []
    class2 = []

    for i in range(ypredproba_all_class_0.shape[0]):
        #         print(i)
        temp_c0 = temp_c1 = temp_c2 = 0
        for j in range(confmatrix.shape[0]):

            temp_c0 += (
                ypredproba_all_class_0.iloc[i][j] * confmatrix.iloc[j][0]
                + ypredproba_all_class_1.iloc[i][j] * confmatrix.iloc[j][1]
                + ypredproba_all_class_2.iloc[i][j] * confmatrix.iloc[j][2]
            )

            temp_c1 += (
                ypredproba_all_class_0.iloc[i][j] * confmatrix.iloc[j][3]
                + ypredproba_all_class_1.iloc[i][j] * confmatrix.iloc[j][4]
                + ypredproba_all_class_2.iloc[i][j] * confmatrix.iloc[j][5]
            )

            temp_c2 += (
                ypredproba_all_class_0.iloc[i][j] * confmatrix.iloc[j][6]
                + ypredproba_all_class_1.iloc[i][j] * confmatrix.iloc[j][7]
                + ypredproba_all_class_2.iloc[i][j] * confmatrix.iloc[j][8]
            )

        class0.append(temp_c0)
        class1.append(temp_c1)
        class2.append(temp_c2)

    finalval = np.argmax(np.column_stack((class0, class1, class2)), axis=1)

    print(m.accuracy_score(ytest, finalval))
    FinalAccuracy.append(m.accuracy_score(ytest, finalval))
    FinalPrecision.append(m.precision_score(ytest, finalval, average="weighted"))
    FinalRecall.append(m.recall_score(ytest, finalval, average="weighted"))
    FinalF1.append(m.f1_score(ytest, finalval, average="weighted"))

0
0.7468871595330739
1
0.7464980544747082
2
0.7529182879377432
3
0.7443082311733801
4
0.7517026658883051
5
0.750340533177661
6
0.7470324965946682
7
0.7499513524031913
8
0.7585133294415256
9
0.7583187390542907


<IPython.core.display.Javascript object>

In [8]:
np.mean(FinalAccuracy)

0.7503052735646928

<IPython.core.display.Javascript object>

In [9]:
np.std(FinalAccuracy)

0.004522995528740559

<IPython.core.display.Javascript object>

In [22]:
print(
    "Accuracy: %0.2f (+/- %0.3f) [%s] \n [%s]"
    % (np.array(FinalAccuracy).mean(), np.array(FinalAccuracy).std(), "", FinalAccuracy)
)

Accuracy: 0.75 (+/- 0.005) [] 
 [[0.7468871595330739, 0.7468871595330739, 0.7464980544747082, 0.7529182879377432, 0.7443082311733801, 0.7517026658883051, 0.750340533177661, 0.7470324965946682, 0.7499513524031913, 0.7585133294415256, 0.7583187390542907]]


<IPython.core.display.Javascript object>

In [21]:
print(
    "FinalPrecision: %0.3f (+/- %0.2f) [%s] \n [%s]"
    % (
        np.array(FinalPrecision).mean(),
        np.array(FinalPrecision).std(),
        "",
        FinalPrecision,
    )
)

FinalPrecision: 0.748 (+/- 0.00) [] 
 [[0.7434836715840674, 0.7454755888693235, 0.7491588264452427, 0.7408070702360673, 0.7495761527502646, 0.7493791642638451, 0.7425270419244586, 0.7468413352777094, 0.754814197995875, 0.75408394748165]]


<IPython.core.display.Javascript object>

In [20]:
print(
    "FinalRecall: %0.2f (+/- %0.3f) [%s] \n [%s]"
    % (np.array(FinalRecall).mean(), np.array(FinalRecall).std(), "", FinalRecall)
)

FinalRecall: 0.75 (+/- 0.005) [] 
 [[0.7468871595330739, 0.7464980544747082, 0.7529182879377432, 0.7443082311733801, 0.7517026658883051, 0.750340533177661, 0.7470324965946682, 0.7499513524031913, 0.7585133294415256, 0.7583187390542907]]


<IPython.core.display.Javascript object>

In [19]:
print(
    "FinalF1: %0.2f (+/- %0.3f) [%s] \n [%s]"
    % (np.array(FinalF1).mean(), np.array(FinalF1).std(), "", FinalF1)
)

FinalF1: 0.75 (+/- 0.004) [] 
 [[0.7447981408528206, 0.7458107093412706, 0.7504058018018133, 0.7420269481103574, 0.7504511879455151, 0.7498436646102544, 0.7434085021049721, 0.7481289602545091, 0.7555908934963916, 0.7552740013097564]]


<IPython.core.display.Javascript object>

7 Genetic algorithm
====

In [ ]:
import calculateWeightUsingGa2 as aresult


In [ ]:
weightvalga

In [ ]:
for cv_i in range(10):

    print(cv_i)

    acc = pd.read_csv("cv_resulsts/CV_" + str(cv_i) + "_acc.txt")
    ytest = pd.read_csv("cv_resulsts/CV_" + str(cv_i) + "_tempytest.txt")

    ypredproba_all_class_0 = pd.read_csv(
        "cv_resulsts/CV_" + str(cv_i) + "_ypredproba_all_class_0.txt"
    ).transpose()
    ypredproba_all_class_1 = pd.read_csv(
        "cv_resulsts/CV_" + str(cv_i) + "_ypredproba_all_class_1.txt"
    ).transpose()
    ypredproba_all_class_2 = pd.read_csv(
        "cv_resulsts/CV_" + str(cv_i) + "_ypredproba_all_class_2.txt"
    ).transpose()
    confmatrix = pd.read_csv("cv_resulsts/CV_" + str(cv_i) + "_confmatrix.txt")

    
    weightvalga = aresult.getbestvalues(acc)

    tempacc = pd.DataFrame(np.array(acc), columns=["acc"])
    indexs = tempacc.index
    len(indexs)

    ypredproba_all_class_0 = ypredproba_all_class_0.iloc[:, indexs]
    ypredproba_all_class_1 = ypredproba_all_class_1.iloc[:, indexs]
    ypredproba_all_class_2 = ypredproba_all_class_2.iloc[:, indexs]

    ypredproba_all_class_0.columns = np.arange(indexs.shape[0])
    ypredproba_all_class_1.columns = np.arange(indexs.shape[0])
    ypredproba_all_class_2.columns = np.arange(indexs.shape[0])

    confmatrix = confmatrix.iloc[indexs, :].reset_index(drop=True)

    class0 = []
    class1 = []
    class2 = []

    for i in range(ypredproba_all_class_0.shape[0]):
        #         print(i)
        temp_c0 = temp_c1 = temp_c2 = 0
        for j in range(confmatrix.shape[0]):

            temp_c0 += (
                ypredproba_all_class_0.iloc[i][j] * confmatrix.iloc[j][0]
                + ypredproba_all_class_1.iloc[i][j] * confmatrix.iloc[j][1]
                + ypredproba_all_class_2.iloc[i][j] * confmatrix.iloc[j][2]
            )

            temp_c1 += (
                ypredproba_all_class_0.iloc[i][j] * confmatrix.iloc[j][3]
                + ypredproba_all_class_1.iloc[i][j] * confmatrix.iloc[j][4]
                + ypredproba_all_class_2.iloc[i][j] * confmatrix.iloc[j][5]
            )

            temp_c2 += (
                ypredproba_all_class_0.iloc[i][j] * confmatrix.iloc[j][6]
                + ypredproba_all_class_1.iloc[i][j] * confmatrix.iloc[j][7]
                + ypredproba_all_class_2.iloc[i][j] * confmatrix.iloc[j][8]
            )

        class0.append(temp_c0)
        class1.append(temp_c1)
        class2.append(temp_c2)

    finalval = np.argmax(np.column_stack((class0, class1, class2)), axis=1)

    print(m.accuracy_score(ytest, finalval))
    FinalAccuracy.append(m.accuracy_score(ytest, finalval))